In [1]:
import numpy as np
import csv
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import u_V
from random import uniform
from time import time
import gc
import os

# Add progress bar support with fallback
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable, *args, **kwargs):
        return iterable

logger = Logging.setup_logging()

BATCH_SIZE = 5000  # Number of MOSFETs per run
PARAMS_FILE = 'mosfet_params_level3_v11.csv'  # Updated filename for clarity
MEASUREMENTS_FILE = 'measurements_level3_v11.csv'  # Updated filename

def generate_random_level3_model():
    """Generates a random Level 3 NMOS model (enhancement mode) with performance similar to Level 1."""
    return {
        'W': uniform(1e-6, 50e-6),       # 1 µm - 50 µm
        'L': uniform(2e-6, 10e-6),       # 2 µm - 10 µm
        'KP': uniform(50e-6, 500e-6),    # 50 µA/V² - 500 µA/V²
        'VTO': uniform(0.3, 1.2),        # 0.3 V - 1.2 V
        'PHI': 0.8,                      # Fixed at 0.8 V
        'GAMMA': uniform(0.2, 0.8),      # V⁰·⁵
        'THETA': uniform(0.05, 0.2),     # V⁻¹
        'ETA': uniform(0.01, 0.1),       # DIBL
        'LAMBDA': uniform(0.01, 0.03),   # V⁻¹ (centers on 0.02)
        'U0': uniform(200, 600),         # cm²/V·s
        'UEXP': uniform(0.1, 0.3),       # Mobility exponent
        'UCRIT': uniform(1e4, 5e4),      # V/cm
        'RD': uniform(50, 500),          # Ω
        'RS': uniform(50, 500),          # Ω
        'CBD': 10**uniform(-12, -10),    # F
        'CBS': 10**uniform(-12, -10),    # F
        'CGDO': uniform(1e-12, 5e-12),   # F
        'CGSO': uniform(1e-12, 5e-12),   # F
        'CGBO': uniform(0.5e-12, 2e-12), # F
        'CJ': uniform(1e-4, 5e-4),       # F/m²
        'CJSW': uniform(1e-10, 5e-10),   # F/m
        'IS': 10**uniform(-14, -12),     # A
        'PB': uniform(0.7, 0.9),         # V
        'MJ': uniform(0.3, 0.5),         # Bottom grading
        'MJSW': uniform(0.2, 0.4),       # Sidewall grading
        'TOX': uniform(10e-9, 50e-9),    # m (10–50 nm)
        'XJ': uniform(0.1e-6, 0.5e-6)    # m (0.1–0.5 µm)
    }

def build_nmos_circuit(model_params):
    circuit = Circuit('Random_Level3_NMOS_Test')
    circuit.model('NMOSMOD', 'NMOS', LEVEL=3, **model_params)  # Updated to LEVEL=3
    circuit.MOSFET('M1', 'drain', 'gate', 'source', 'source', model='NMOSMOD')
    circuit.V('dd', 'drain', circuit.gnd, 0@u_V)
    circuit.R('SRC', 'source', circuit.gnd, 0)
    circuit.V('in', 'gate', circuit.gnd, 0@u_V)
    return circuit

def get_output_characteristic(model_params, gate_voltages, vds_start=0.0, vds_stop=5.0, vds_step=0.1):
    circuit = build_nmos_circuit(model_params)
    simulator = circuit.simulator(temperature=25, nominal_temperature=25)
    results = []
    meas_index = 0  # Initialize measurement index

    for vgs in gate_voltages:
        circuit['Vin'].dc_value = vgs
        analysis = simulator.dc(Vdd=slice(vds_start, vds_stop, vds_step))
        vds_array = np.array(analysis.nodes['drain'])
        id_array = -np.array(analysis.branches['vdd'])
        
        # Append measurement index to each data point
        for vds, id_val in zip(vds_array, id_array):
            results.append((meas_index, vgs, vds, id_val))
            meas_index += 1  # Increment measurement index

    return results

def get_last_mosfet_id():
    if not os.path.exists(PARAMS_FILE):
        return 0  # No previous data exists
    with open(PARAMS_FILE, 'r') as file:
        lines = file.readlines()
        if len(lines) <= 1:
            return 0
        return int(lines[-1].split(',')[0])

def main():
    start_time = time()
    gate_voltages = np.arange(0.0, 5.5, 0.5)
    last_id = get_last_mosfet_id()
    print(f"Last MOSFET ID: {last_id}")
    
    # Open CSV files in append mode
    with open(PARAMS_FILE, 'a', newline='') as params_file, \
         open(MEASUREMENTS_FILE, 'a', newline='') as measurements_file:
        
        params_writer = csv.writer(params_file)
        measurements_writer = csv.writer(measurements_file)
        
        # If the files are new, write headers with all Level 3 parameters
        if last_id == 0:
            params_writer.writerow([
                "MOSFET_ID", "L", "W", "KP", "VTO", "PHI", "GAMMA", "THETA", "ETA", "LAMBDA",
                "U0", "UEXP", "UCRIT", "RD", "RS", "CBD", "CBS", "CGDO", "CGSO", "CGBO",
                "CJ", "CJSW", "IS", "PB", "MJ", "MJSW", "TOX", "XJ"
            ])
            measurements_writer.writerow(["MOSFET_ID", "meas_index", "VGS", "VDS", "ID"])

        mosfet_iterator = tqdm(range(last_id + 1, last_id + BATCH_SIZE + 1), 
                               desc="Generating MOSFETs", 
                               total=BATCH_SIZE, 
                               unit="device", 
                               dynamic_ncols=True)
        
        try:
            for mosfet_id in mosfet_iterator:
                model_params = generate_random_level3_model()
                
                # Write all Level 3 parameters
                params_writer.writerow([
                    mosfet_id,
                    model_params['L'],
                    model_params['W'],
                    model_params['KP'],
                    model_params['VTO'],
                    model_params['PHI'],
                    model_params['GAMMA'],
                    model_params['THETA'],
                    model_params['ETA'],
                    model_params['LAMBDA'],
                    model_params['U0'],
                    model_params['UEXP'],
                    model_params['UCRIT'],
                    model_params['RD'],
                    model_params['RS'],
                    model_params['CBD'],
                    model_params['CBS'],
                    model_params['CGDO'],
                    model_params['CGSO'],
                    model_params['CGBO'],
                    model_params['CJ'],
                    model_params['CJSW'],
                    model_params['IS'],
                    model_params['PB'],
                    model_params['MJ'],
                    model_params['MJSW'],
                    model_params['TOX'],
                    model_params['XJ']
                ])
                
                # Generate and write measurements
                measurements = get_output_characteristic(
                    model_params,
                    gate_voltages=gate_voltages
                )
                
                for meas_index, vgs, vds, id_val in measurements:
                    measurements_writer.writerow([mosfet_id, meas_index, vgs, vds, id_val])
                
                # Flush periodically
                if mosfet_id % 100 == 0:
                    params_file.flush()
                    measurements_file.flush()
                    gc.collect()
            
        except KeyboardInterrupt:
            print("\nProcess interrupted. Partial data saved.")
    
    total_time = time() - start_time
    print(f"Completed {BATCH_SIZE} MOSFETs in {total_time:.2f}s")
    print(f"Rate: {BATCH_SIZE/total_time:.2f} devices/sec")

if __name__ == "__main__":
    main()

Last MOSFET ID: 50000


Generating MOSFETs: 100%|██████████| 5000/5000 [01:49<00:00, 45.79device/s]

Completed 5000 MOSFETs in 109.29s
Rate: 45.75 devices/sec
